In [48]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

### Defining Setup Time and Hourly Yield Constants

In [49]:
T = pd.read_csv('setup_constants.csv')
R = pd.read_csv('yield_constants.csv')

In [50]:
T.index += 1
R.index += 1

### Defining Weekly Demand Constants

In [51]:
D = pd.read_csv('weekly_demands.csv')

In [52]:
D.index += 1

# Defining Problem

In [53]:
model = gp.Model('base_problem_max')

## Variables

### Adding Binary Setup Variables

In [54]:
S = []
for i in range(5):
    S_i = []
    for j in range(5):
        S_i.append(model.addVar(vtype=GRB.BINARY, name=f"S_{i}_{j}", lb=0))
    S.append(S_i)

### Adding Binary Overtime Setup Variables

In [55]:
Sp = []
for i in range(5):
    Sp_i = []
    for j in range(5):
        Sp_i.append(model.addVar(vtype=GRB.BINARY, name=f"Sp_{i}_{j}", lb=0))
    Sp.append(Sp_i)

### Adding Production Variables

In [56]:
P = []
for i in range(5):
    P_i = []
    for j in range(5):
        P_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"P_{i}_{j}", lb=0))
    P.append(P_i)

### Adding Overtime Production Variables

In [57]:
Pp = []
for i in range(5):
    Pp_i = []
    for j in range(5):
        Pp_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"Pp_{i}_{j}", lb=0))
    Pp.append(Pp_i)

In [58]:
t = model.addVar(vtype=GRB.CONTINUOUS, name="t", lb=0)

## Constraints

### Demand Constraints

In [59]:
for i in range(5):
    constr_expr = gp.quicksum(P[j][i]*R.iloc[j,i] + Pp[j][i]*R.iloc[j,i] for j in range(5))
    model.addConstr(constr_expr >= D.iloc[0][i], name=f'Demand_Prod_{i+1}')

/var/folders/w_/z02fg3l56_jfc9tys40hqtdm0000gn/T/ipykernel_16070/2297407209.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model.addConstr(constr_expr >= D.iloc[0][i], name=f'Demand_Prod_{i+1}')


### Normal Production Time Constraints

In [60]:
for i in range(5):
    constr_expr = gp.quicksum(S[i][j]*T.iloc[i,j] + P[i][j] for j in range(5))
    model.addConstr(constr_expr <= 120, name=f'Prod_Hrs_Mach_{i+1}')

### Overtime Production Time Constraints

In [61]:
for i in range(5):
    constr_expr = gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5))
    model.addConstr(constr_expr <= 48, name=f'OT_Prod_Hrs_Mach_{i+1}')

### Production Setup Linkage Constants
- Regular Time and Overtime are both included here

In [62]:
for i in range(5):
    for j in range(5):
        model.addConstr(P[i][j]*R.iloc[i,j] <= 10000*S[i][j], name=f'SetupProdLinkage_Mach_{i+1}_Prod_{j+1}')
        model.addConstr(Pp[i][j]*R.iloc[i,j] <= 10000*(S[i][j]+Sp[i][j]), name=f'OTSetupProdLinkage_Mach_{i+1}_Prod_{j+1}')

### Maximisation Constraints
- Since we're trying to convert the max(overtime[i]) function - which is not a linear function - to a linear function for the purpose of solving it as an LP, we add additional constraints.

In [63]:
for i in range(5):
    model.addConstr(gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5)) <= t, name=f'MaxConstr_{i}')

## Objective Function

In [64]:
obj_parts = []
for i in range(5):
    obj_parts.append(gp.quicksum(Sp[i][j]*T.iloc[i,j] + Pp[i][j] for j in range(5))*30)

In [65]:
model.setObjective(t*40 + gp.quicksum(obj_parts),GRB.MINIMIZE)

In [66]:
model.update()

In [67]:
model.getObjective()

<gurobi.LinExpr: 240.0 Sp_0_0 + 240.0 Sp_0_3 + 300.0 Sp_1_0 + 240.0 Sp_1_1 + 300.0 Sp_2_1 + 720.0 Sp_2_4 + 240.0 Sp_3_1 + 360.0 Sp_3_2 + 240.0 Sp_4_3 + 600.0 Sp_4_4 + 30.0 Pp_0_0 + 30.0 Pp_0_1 + 30.0 Pp_0_2 + 30.0 Pp_0_3 + 30.0 Pp_0_4 + 30.0 Pp_1_0 + 30.0 Pp_1_1 + 30.0 Pp_1_2 + 30.0 Pp_1_3 + 30.0 Pp_1_4 + 30.0 Pp_2_0 + 30.0 Pp_2_1 + 30.0 Pp_2_2 + 30.0 Pp_2_3 + 30.0 Pp_2_4 + 30.0 Pp_3_0 + 30.0 Pp_3_1 + 30.0 Pp_3_2 + 30.0 Pp_3_3 + 30.0 Pp_3_4 + 30.0 Pp_4_0 + 30.0 Pp_4_1 + 30.0 Pp_4_2 + 30.0 Pp_4_3 + 30.0 Pp_4_4 + 40.0 t>

In [68]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 70 rows, 101 columns and 225 nonzeros
Model fingerprint: 0x3d6d93ff
Variable types: 51 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [3e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 4e+03]
Found heuristic solution: objective 6480.0000000
Presolve removed 39 rows and 69 columns
Presolve time: 0.00s
Presolved: 31 rows, 32 columns, 104 nonzeros
Variable types: 17 continuous, 15 integer (15 binary)

Root relaxation: objective 4.461168e+03, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4461.16809    0    6 6480.00000 4461.16809  

In [69]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(S[i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 8 0 0 8 0 
Machine 2: 10 0 0 0 0 
Machine 3: 0 10 0 0 0 
Machine 4: 0 8 12 0 0 
Machine 5: 0 0 0 8 20 


In [70]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sp[i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [71]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(P[i][j].X)), end=" ")
    print("")

Machine 1: 49.583333333333336 0.0 0.0 54.416666666666664 0.0 
Machine 2: 110.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 110.0 0.0 0.0 0.0 
Machine 4: 0.0 0.0 100.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 35.403263403263416 56.59673659673659 


In [72]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pp[i][j].X)), end=" ")
    print("")

Machine 1: 0.0 0.0 0.0 12.744172494172476 0.0 
Machine 2: 0.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 36.73659673659674 0.0 0.0 0.0 
Machine 4: 0.0 30.06993006993007 6.666666666666667 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 36.736596736596745 


In [73]:
print('Final Objective Value: ', model.ObjVal)

Final Objective Value:  5158.08275058275
